# Algoritmo Apriori
Propuesto por Agrawal y Srikant en 1994. Este tiene como objetivo encontrar itemsets frecuentes dentro de una base de datos y generar reglas de asociación bajo determinados umbrales de soporte y confianza.

Autores:
- Espinoza Champi Israel Enrique
- Vilcahuaman Caceres Miguel

## Librerías necesarias

In [1]:
import itertools # combinaciones y permutaciones
import numpy as np # leer el dataset
import pandas as pd # mostrar los resultados

## Base de datos
La base de datos a utilizar corresponde a múltiples playlists de la plataforma spotify creadas
por usuarios de esta.

In [2]:
# cargamos la base de datos
load = np.load('spotify.npy', allow_pickle = True)
trans_dic = load.tolist()
trans = list(trans_dic.values())
#trans = load.tolist()

In [3]:
# variables globales
data = sorted(trans, key=lambda x:len(x)) # ordenado por longitud de sublistas
total_trans = len(data) # total de transacciones
dic_support = dict() # diccionario de supports
it_cont = 0 # desde donde iterar, ignorando subconjuntos de menor tamaño

## Funciones 

Función **getListItems**

Parámetros:
- data: lista de listas

Salida:
- lista con valores únicos de todas las listas de data

In [4]:
# obtener todos los items diferentes de data
def getListItems(data):
	list_items = set()
	for list_ in data:
		list_items = list_items | set(list_)
	return list(list_items)

Función **first_items**

Parámetros:
- min_support: el minimo soporte que debe tener cada primer candidato

Salida:
- arreglo de los primeros candidatos

In [12]:
# primeros candidatos
def first_items(min_support):
    dic_t = dict()
    # contamos el total de apariciones de cada item
    for itemset in data:
        for item in itemset:
            if item not in dic_t: dic_t[item] = 1
            else: dic_t[item] += 1
    first = []
    for item in dic_t: 
        # agregamos los items que alcanzan el support al diccionario
        supp = dic_t[item]/total_trans
        if supp >= min_support:
            first.append([item])
            dic_support[str(set([item]))] = round(supp, 4)
    return first

Función **powerSet**

Parámetros:
- c: lista de items

Salida:
- conjunto potencia de c

In [6]:
def powerSet(c):
    #Calcula y devuelve el conjunto potencia del conjunto c.
    if len(c) == 0:
        return [[]]
    r = powerSet(c[:-1])
    return r + [s + [c[-1]] for s in r]

Función **support**

Parametros:
- sub_set: subconjunto de items

Salida:
- Usa las variables globales de: `dic_support` para verificar si ya se calculó el support de `sub_set`, si no itera las transacciones desde `it_cont`, almacena y retorna el support de `sub_set`



In [7]:
# retorna el support de un subconjunto
def support(sub_set):
	sub_set = set(sub_set)
	# buscamos en el diccionario
	if(str(sub_set) not in dic_support):
		cont = 0
		for k in range(it_cont,total_trans):
			if set(sub_set) <= set(data[k]):
				cont += 1
		dic_support[str(sub_set)] = round(cont/total_trans,4)
	return dic_support[str(sub_set)]

Función **generate_apriori**

Parámetros:
- playlist: base de datos
- Lk: lista de posibles candidatos
- min_support: soporte mínimo que debe superar cada candidato
- cont: de que tamaño se formaran los subconjuntos

Salida:
- arreglo de subconjuntos que superan el `min_support`

In [8]:
def generate_apriori(playlist, Lk, min_support, cont):
    global it_cont
    arr_s = [] # array de salida de itemsets frecuentes
    for k in range(it_cont, total_trans):
        if len(playlist[it_cont]) < cont:
            it_cont += 1
        else: break
    Lk = getListItems(Lk)
    # combinaciones en Lk de "cont" elementos
    arr_C = list(itertools.combinations(Lk,cont))
    for subSet in arr_C:
        cont = 0
        for k in range(it_cont, total_trans):
            if set(subSet) <= set(playlist[k]):
                cont+=1 # contamos la apariciones de cada subconjunto
        cont = cont / total_trans
        if cont >= min_support:
            arr_s.append(list(subSet))
            dic_support[str(set(subSet))] = round(cont, 4)
    return arr_s


## Funciones principales

Función **get_frequent_itemsets**

Parámetros:
- playlist: base de datos
- min_support: soporte mínimo que debe superar cada itemset frecuente

Salida:
- lista de itemsets frecuentes

In [9]:
def get_frequent_itemsets(playlist, min_support):
    global it_cont
    it_cont = 0
    Lk = first_items(min_support) # lista primeros candidatos
    frequent_items = []
    Ck = [] # itemsets que superan el min_support
    cont = 2 # iniciamos con subconjuntos de 1 elemento
    while(True):
        if cont == 2:
            frequent_items = generate_apriori(playlist, Lk, min_support, cont)
        else:
            Ck = frequent_items.copy()
            frequent_items = generate_apriori(playlist, Ck, min_support, cont)
        if len(frequent_items)==0: break
        cont += 1
    return Ck

Función **generate_association_rules**

Parámetros:
- frequent_itemsets: lista de los itemsets frecuentes
- confidence: confianza mínima que debe superar cada regla
- lift: lift mínimo que debe superar cada regla

Salida:
- diccionario de reglas de asociación de la forma `{regla} : [soporte de la regla, confianza y lift]`

In [10]:
def generate_association_rules(frequent_itemsets, confidence = 0, lift = 0):
	rules = dict()
	for itemset in frequent_itemsets:
		arr_Pot = powerSet(itemset)
		arr_Pot = arr_Pot[1:] # sin el conjunto vacio
		arr_Pot = arr_Pot[:-1] # sin el propio conjunto
		# para los subconjuntos buscamos las reglas de asociacion
		arr_Rules = list(itertools.permutations(arr_Pot,2))
		for rule in arr_Rules:
			A = set(rule[0])
			B = set(rule[1]) # A => B
			AintB = A & B # interseccion de A y B
			if len(AintB) == 0 :
				key_rule = str(A) + " => " + str(B)
				if key_rule not in rules:
					# support( A => B ) = support(A U B)
					supp_rule = support( A | B )
					# confidence(A => B) = support(A U B) / support(A)
					supp_A = support( A )
					confidence_rule = supp_rule / supp_A
					# lift(A => B) = confidence(A => B) / support(B)
					supp_B = support(B)
					lift_rule = confidence_rule / supp_B
					# buscamos las reglas que cumple con >= confidence y >= lift
					if confidence_rule >= confidence and lift_rule >=lift:
						rules[key_rule] = [round(supp_rule,4), round(confidence_rule,4), round(lift_rule,4)]
	return rules

## Ejecucion del algoritmo

In [13]:
# obtenemos la lista de subconjuntos más frecuentes
min_support = 0.012 #debe aparecer en almenos 1.2% de los itemsets
itemsets_frequent = get_frequent_itemsets(data, min_support)

In [14]:
# mostramos los itemsfrecuentes encontrados
itemsets_frequent

[['XO TOUR Llif3', 'Mask Off', 'HUMBLE.'],
 ['XO TOUR Llif3', 'HUMBLE.', 'Congratulations'],
 ['Mask Off', 'HUMBLE.', 'Congratulations']]

In [54]:
# obtenemos las reglas de asociacion
confidence = 0.5
lift = 10
association_rules = generate_association_rules(itemsets_frequent, confidence, lift)

### Mostramos las reglas obtenidas en un data frame

In [55]:
df = pd.DataFrame() # creamos el dataframe
# añadimos columnas
df['Rule'] = None
df['Support'] = None
df['Confidence'] = None
df['Lift'] = None

In [56]:
# agregamos las reglas de asociacion halladas
for key in association_rules:
    val = association_rules[key]
    new_row = { 'Rule'      : key, 
                'Support'   : val[0],
                'Confidence': val[1],
                'Lift'      : val[2]}
    df = df.append(new_row, ignore_index=True)


In [57]:
df

,Rule,Support,Confidence,Lift
0,{'XO TOUR Llif3'} => {'HUMBLE.'},0.0204,0.6182,13.1528
1,{'Mask Off'} => {'XO TOUR Llif3'},0.0163,0.5158,15.6310
2,{'Mask Off'} => {'HUMBLE.'},0.0204,0.6456,13.7355
3,"{'Mask Off', 'XO TOUR Llif3'} => {'HUMBLE.'}",0.0131,0.8037,17.0996
4,"{'XO TOUR Llif3', 'HUMBLE.'} => {'Mask Off'}",0.0131,0.6422,20.3214
5,"{'Mask Off', 'HUMBLE.'} => {'XO TOUR Llif3'}",0.0131,0.6422,19.4593
6,{'XO TOUR Llif3'} => {'Congratulations'},0.0179,0.5424,13.2299
7,"{'XO TOUR Llif3', 'HUMBLE.'} => {'Congratulati...",0.0128,0.6275,15.3037
8,{'Congratulations'} => {'HUMBLE.'},0.0214,0.5220,11.1053
9,"{'XO TOUR Llif3', 'Congratulations'} => {'HUMB...",0.0128,0.7151,15.2145


## Filtramos las mejores 10 reglas de asociación por mayor Confidence

In [53]:
by_confidence = df.sort_values('Confidence',ascending=False)
by_confidence[:10]

,Rule,Support,Confidence,Lift
3,"{'Mask Off', 'XO TOUR Llif3'} => {'HUMBLE.'}",0.0131,0.8037,17.0996
13,"{'Mask Off', 'Congratulations'} => {'HUMBLE.'}",0.0121,0.7469,15.8918
9,"{'XO TOUR Llif3', 'Congratulations'} => {'HUMB...",0.0128,0.7151,15.2145
2,{'Mask Off'} => {'HUMBLE.'},0.0204,0.6456,13.7355
4,"{'XO TOUR Llif3', 'HUMBLE.'} => {'Mask Off'}",0.0131,0.6422,20.3214
5,"{'Mask Off', 'HUMBLE.'} => {'XO TOUR Llif3'}",0.0131,0.6422,19.4593
7,"{'XO TOUR Llif3', 'HUMBLE.'} => {'Congratulati...",0.0128,0.6275,15.3037
0,{'XO TOUR Llif3'} => {'HUMBLE.'},0.0204,0.6182,13.1528
10,"{'HUMBLE.', 'Congratulations'} => {'XO TOUR Ll...",0.0128,0.5981,18.1252
12,"{'Mask Off', 'HUMBLE.'} => {'Congratulations'}",0.0121,0.5931,14.4668


## Explicación de las reglas obtenidas

In [63]:
by_confidence[0:1]

,Rule,Support,Confidence,Lift
3,"{'Mask Off', 'XO TOUR Llif3'} => {'HUMBLE.'}",0.0131,0.8037,17.0996


- Estas 3 canciones ocurren en el 1.31% de las playlist en este dataset,
lo que quiere decir que estan presentes en 131 playlist de las 10 000 que
hay en nuestro dataset.
- Además, podemos decir que si en una playlist estan "Mask Off" y "XO TOUR Llif3"
pueden aparecer junto a "HUMBLE" con una seguridad del 80.37 %.
- Por ultimo, al tener un lift bastante alejado de 1 (17.0996)
podemos decir que la regla representa un valor real, es decir, cuando en una
playlist estas juntas las canciones "Mask Off" y "XO TOUR Llif3" aumenta la
probabilidad de que tambien este "HUMBLE".

In [64]:
by_confidence[1:2]

,Rule,Support,Confidence,Lift
13,"{'Mask Off', 'Congratulations'} => {'HUMBLE.'}",0.0121,0.7469,15.8918


- Estas 3 canciones ocurren en el 1.21% de las playlist en este dataset,
lo que quiere decir que estan presentes en 121 playlist de las 10 000 que 
hay en nuestro dataset,
- Además, podemos decir que si en una playlist estan "Mask Off" y 
"Congratulations" pueden aparecer junto a "HUMBLE" con una seguridad del 74.69 %.
- Por ultimo, al tener un lift tambien bastante alejado de 1, 
podemos decir que la regla representa un valor real, es decir, cuando en una
playlist estan juntas "Mask Off" y "Congratulations", aumenta la probabilidad
de que tambien este "HUMBLE".

In [66]:
by_confidence[9:10]

,Rule,Support,Confidence,Lift
12,"{'Mask Off', 'HUMBLE.'} => {'Congratulations'}",0.0121,0.5931,14.4668


- En esta regla tenemos que la probalidad de que una persona escuche "Congratulations" habiendo escuchado "Mask Off" y "HUMBLE." es de 59%.
- Además, la probabilidad de escuchar las 3 canciones juntas es 14.4 mayor que escuchar solo "Congratulations".
- En cuanto a los gustos musicales las 3 canciones pertenecen al genero de Hip-hop/rap. 

In [67]:
by_confidence[8:9]

,Rule,Support,Confidence,Lift
10,"{'HUMBLE.', 'Congratulations'} => {'XO TOUR Ll...",0.0128,0.5981,18.1252


- En esta regla tenemos que la probalidad de que una persona escuche "XO TOUR Llif3" habiendo escuchado "HUMBLE. y Congratulations" tambien es de 59%.
- Además, la probabilidad de escuchar las 3 canciones juntas es 18.1 mayor que escuchar solo "XO TOUR Llif3".
- En cuanto a los gustos musicales las 3 canciones pertenecen al genero de Hip-hop/rap, al igual que la regla anterior las 3 canciones pertenecen a diferentes artistas.

### Conclusión
Todos estos resultado coinciden con la realidad de personas cuyos
gustos musicales estan relacionados al hip- hop, y, al pertenecer estas 
canciones a ese genero, tiene mucho sentido que se encuentren juntas en una
playlist.